In [ ]:
# importing Dependencies
import requests
import json
import pandas as pd

In [ ]:
# Define API endpoints and headers
teams_url = "https://api-football-v1.p.rapidapi.com/v3/teams"
stats_url = "https://api-football-v1.p.rapidapi.com/v3/teams/statistics"
headers = {
    "X-RapidAPI-Key": "e745f20ff6mshb3a174087c1d159p11ee9ejsn4221f696691e",
    "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
}

# Define the range of seasons
start_season = 2019
end_season = 2024
league_id = "371"  # Example league ID

# Initialize an empty list to hold all team IDs
all_team_ids = []

# Function to fetch teams for a given season and add their IDs to all_team_ids
def fetch_teams_for_season(season):
    querystring = {"league": league_id, "season": str(season)}
    response = requests.get(teams_url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        data = response.json()
        for team in data['response']:
            team_id = team['team']['id']
            if team_id not in all_team_ids:
                all_team_ids.append(team_id)
    else:
        print(f"Failed to fetch teams for season {season}: {response.status_code}")

# Fetch teams for all seasons
for year in range(start_season, end_season):
    fetch_teams_for_season(year)

# Initialize a list to hold all statistics data
all_stats_data = []

# Function to fetch statistics for a given team and season
def fetch_team_statistics(season, team_id):
    querystring = {"league": league_id, "season": str(season), "team": str(team_id)}
    response = requests.get(stats_url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        data = response.json()
        all_stats_data.append(data)
    else:
        print(f"Failed to fetch stats for team {team_id} in season {season}: {response.status_code}")

# Fetch statistics for all teams in all seasons
for year in range(start_season, end_season):
    for team_id in all_team_ids:
        fetch_team_statistics(year, team_id)

# Print the aggregated statistics data in a formatted way
print(json.dumps(all_stats_data, indent=4))

In [ ]:
#Team data table 
team_data_table = []

#looping through JSON 
for entry in all_stats_data:
    response = entry["response"]
    team_entry = {
        "Team Name": response["team"]["name"],
        "Season": response["league"]["season"],
        "Total Games Played": response["fixtures"]["played"]["total"],
        "Total Wins": response["fixtures"]["wins"]["total"],
        "Total Draws": response["fixtures"]["draws"]["total"],
        "Total Loses": response["fixtures"]["loses"]["total"],
        "Total Goals": response["goals"]["for"]["total"]["total"],
        "Average Goal Scored": response["goals"]["for"]["average"]["total"],
        "Total Goals Conceded": response["goals"]["against"]["total"]["total"],
        "Average Goals Conceded": response["goals"]["against"]["average"]["total"],
        "Biggest Win Streak": response["biggest"]["streak"]["wins"],
        "Biggest Draw Streak": response["biggest"]["streak"]["draws"],
        "Biggest Losing Streak": response["biggest"]["streak"]["loses"],
        "Total Clean Sheet Games": response["clean_sheet"]["total"],
        "Total Games with no goals": response["failed_to_score"]["total"],
        "Total Penalties Scored": response["penalty"]["scored"]["total"],
        "Total Penalties Missed": response["penalty"]["missed"]["total"]
    }

    team_data_table.append(team_entry)


#passing data and defining column names
team_stats_df = pd.DataFrame(team_data_table)

team_stats_df.head()

In [ ]:
#check data types 
# Define dictionary with column names and desired data types
dtypes_dict = {
    "Team Name": str,
    "Season": int,
    "Total Games Played": int,
    "Total Wins": int,
    "Total Draws": int,
    "Total Loses": int,
    "Total Goals": int,
    "Average Goal Scored": float,
    "Total Goals Conceded": int,
    "Average Goals Conceded": float,
    "Biggest Win Streak": int,
    "Biggest Draw Streak": int,
    "Biggest Losing Streak": int,
    "Total Clean Sheet Games": int,
    "Total Games with no goals": int,
    "Total Penalties Scored": int,
    "Total Penalties Missed": int
}

# Change data types of columns
team_stats_1_df = team_stats_df.astype(dtypes_dict)

In [ ]:
# Remove rows where all columns except 'Team Name' and 'Season' are zero
df_filtered = team_stats_1_df.loc[~(team_stats_1_df.drop(columns=['Team Name', 'Season']) == 0).all(axis=1)]

In [ ]:
#Converting First Macedonia League data to Csv
df_filtered.to_csv('First_Macedonia_League_Team_Stats.csv', index=False)